<a href="https://colab.research.google.com/github/sf-rahul/Tensorflow/blob/master/Recurrent_nueral_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**time series forecasting using rnns.**

In [1]:
#rnnn
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import numpy as np
import os
import time

tf.random.set_seed(1)

TensorFlow 2.x selected.


In [0]:
#function to create time series data of length n_steps
#input to the rnn is [batch_size, time steps , dimensionality]
#a time series is cosistting of values having per time step
#value is univariabte then it is calles univariate time series
#value is multivariate then it is calles multivariate time series.
def generate_time_series(batch_size,n_steps):
  freq1,freq2,offset1,offset2 = np.random.rand(4,batch_size,1)#4 random array of shape (batch_size,1)
  time = np.linspace(0,1,n_steps)
  series = 0.5  * np.sin((time-offset1) * (freq1*10+10)) #wave1
  series+= 0.2 * np.sin((time-offset2) * (freq2*20 +20)) #+wave2
  series +=0.1*(np.random.rand(batch_size,n_steps) - 0.5) #+noise
  #print(series)
  #to add new axis to data.
  return series[...,np.newaxis].astype(np.float32)




In [0]:
series = generate_time_series(4,3) #shape(4,2,1)
print(series)
print('--------------')
x,y=series[:3,:2],series[:3,-1]
print(x)
print('------------------')
print(y)


[[[-0.09269179]
  [-0.27487454]
  [ 0.26764813]]

 [[-0.3955232 ]
  [-0.47525486]
  [-0.3857966 ]]

 [[ 0.58213425]
  [ 0.49402696]
  [ 0.30883703]]

 [[ 0.3142075 ]
  [ 0.546498  ]
  [ 0.05040054]]]
--------------
[[[-0.09269179]
  [-0.27487454]]

 [[-0.3955232 ]
  [-0.47525486]]

 [[ 0.58213425]
  [ 0.49402696]]]
------------------
[[ 0.26764813]
 [-0.3857966 ]
 [ 0.30883703]]


In [0]:
#creating training , validation and test sets
n_steps =50 #no of time stamps or steps  in a time series 
series = generate_time_series(10000,n_steps+1)
X_train ,y_train = series[:7000,:n_steps],series[:7000,-1]
X_valid,y_valid = series[7000:9000,:n_steps],series[7000:9000,-1]
X_test,y_test = series[9000:,:n_steps],series[9000:,-1]


In [4]:
print(X_train.shape,y_train.shape)
print(X_valid.shape,y_valid.shape)
print(X_test.shape,y_test.shape)

(7000, 50, 1) (7000, 1)
(2000, 50, 1) (2000, 1)
(1000, 50, 1) (1000, 1)


In [5]:
#Baseline model 1. 
#naive forecasting := predict the last value in each series.
#just pick the last value as predicted value.
y_pred = X_valid[:,-1]
np.mean(tf.keras.losses.mean_squared_error(y_valid,y_pred))

0.020904407

In [0]:
#Baseline model 2
#use a fully connected network.
#the input will be flattene to 50*1 = 50 features(time steps)
#predictin will be simply linear compination of the time step values.
model2 = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape=[50,1]),
                                     tf.keras.layers.Dense(1)])



In [0]:
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 51
Trainable params: 51
Non-trainable params: 0
_________________________________________________________________


In [0]:
model2.compile(optimizer='adam', loss='mse')

In [8]:
model2.fit(x=X_train,y=y_train,epochs=20,validation_data=(X_valid,y_valid))

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 1s 105us/sample - loss: 0.0883 - val_loss: 0.0373
Epoch 2/20
7000/7000 [==============================] - 0s 49us/sample - loss: 0.0255 - val_loss: 0.0185
Epoch 3/20
7000/7000 [==============================] - 0s 50us/sample - loss: 0.0152 - val_loss: 0.0129
Epoch 4/20
7000/7000 [==============================] - 0s 50us/sample - loss: 0.0116 - val_loss: 0.0105
Epoch 5/20
7000/7000 [==============================] - 0s 49us/sample - loss: 0.0097 - val_loss: 0.0091
Epoch 6/20
7000/7000 [==============================] - 0s 50us/sample - loss: 0.0083 - val_loss: 0.0079
Epoch 7/20
7000/7000 [==============================] - 0s 48us/sample - loss: 0.0073 - val_loss: 0.0070
Epoch 8/20
7000/7000 [==============================] - 0s 46us/sample - loss: 0.0065 - val_loss: 0.0062
Epoch 9/20
7000/7000 [==============================] - 0s 48us/sample - loss: 0.0058 - val_loss: 0.0056
Epoch 

**IMPLEMENTING SIMPLE RNN**

In [0]:
SimpleRnn = tf.keras.models.Sequential([
                                        tf.keras.layers.SimpleRNN(1,input_shape=[None,1])
                       ])

In [14]:
SimpleRnn.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [15]:
SimpleRnn.compile(optimizer='adam',loss='mse')
SimpleRnn.fit(x=X_train,y=y_train,epochs=20,validation_data=(X_valid,y_valid))

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 3s 387us/sample - loss: 0.2219 - val_loss: 0.2026
Epoch 2/20
7000/7000 [==============================] - 2s 281us/sample - loss: 0.1850 - val_loss: 0.1682
Epoch 3/20
7000/7000 [==============================] - 2s 278us/sample - loss: 0.1554 - val_loss: 0.1413
Epoch 4/20
7000/7000 [==============================] - 2s 283us/sample - loss: 0.1316 - val_loss: 0.1186
Epoch 5/20
7000/7000 [==============================] - 2s 285us/sample - loss: 0.1089 - val_loss: 0.0963
Epoch 6/20
7000/7000 [==============================] - 2s 286us/sample - loss: 0.0875 - val_loss: 0.0769
Epoch 7/20
7000/7000 [==============================] - 2s 283us/sample - loss: 0.0690 - val_loss: 0.0601
Epoch 8/20
7000/7000 [==============================] - 2s 284us/sample - loss: 0.0533 - val_loss: 0.0459
Epoch 9/20
7000/7000 [==============================] - 2s 281us/sample - loss: 0.0402 - val_loss: 0.034

In [0]:
#The simple rnn is better than the naive one but not than the fully connected one.

**DEEP RNNS**
Stack multiple layers of the cells.

In [0]:
DeepRnn = tf.keras.models.Sequential([
                                 tf.keras.layers.SimpleRNN(20,return_sequences=True,input_shape=[None,1]),
                                 tf.keras.layers.SimpleRNN(20,return_sequences=True),
                                 tf.keras.layers.SimpleRNN(1)

 ])

In [17]:
DeepRnn.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_3 (SimpleRNN)     (None, None, 20)          440       
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, None, 20)          820       
_________________________________________________________________
simple_rnn_5 (SimpleRNN)     (None, 1)                 22        
Total params: 1,282
Trainable params: 1,282
Non-trainable params: 0
_________________________________________________________________
